In [ ]:
from groq import Groq
import pandas as pd
import json

In [ ]:
file_path = '../data/test.json'

with open(file_path, 'r') as file:
    data = json.load(file)

api_key="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"
client = Groq(api_key=api_key)

def get_model_response(prompt):
    system_message = {
        'role': 'system',
        'content': "You're a helpful assistant with expertise in understanding the concepts of a given claim and writing subclaims that decompose the main claim well."
    }
    user_message = {
        'role': 'user',
        'content': prompt
    }

    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[system_message, user_message],
        temperature=0,
        top_p = 0.8,
        max_tokens=512,
        seed = 42
    )

    response = completion.choices[0].message.content
    return response

claims = []
key_points = []
counter = 0

for item in data:
    claim = item['claim']
    claims.append(claim)
    prompt = """Based on the given claim, your task is to extract several distinct key points (2 to 4, depending on the claim's length and complexity) without paraphrasing, in the format of short sentences. Focus on key points that support the main intent of the claim, rather than unnecessary details. Then, only if the number of key points is more than two, identify the pairs of key points whose combination leads to new and richer key points, and return a single coherent short text as a representation of each combination without paraphrasing. Provide your response explicitly in the format of {"key_points": [], "combined_key_points": []}.\n\n""" + f"Claim: {claim}"

    response = get_model_response(prompt)

    start_idx = response.find('{')
    end_idx = response.rfind('}') + 1

    if start_idx != -1 and end_idx != -1:
        try:
            response_dict = json.loads(response[start_idx:end_idx])
        except Exception as e:
            key_points.append({})
            print("The generated format is incorrect.")
            print(claim)
            print("-----")
            continue
    else:
        key_points.append({})
        print("The generated format is incorrect.")
        print(claim)
        print("-----")
        continue

    result = {"main": "", "combined": [], "single": []} 

    result["main"] = claim
    result["combined"] = response_dict["combined_key_points"]
    result["single"] = response_dict["key_points"]

    key_points.append(result)
    counter += 1
    if counter % 250 == 0:
        print(counter)

df = pd.DataFrame({
    'Claims': claims,
    'Key Points': key_points
})

df.to_csv('../data/keypoints.csv', index=False)